In [18]:
import json
import geopandas as gpd
from shapely.geometry import MultiPoint
import plotly.express as px
import random
import folium
import folium.plugins as plugins
from shapely.geometry import MultiPoint

In [19]:
# geojson data
admin_shp_path ='../data/SHP/Administrasi.shp'
# GeoDataFrame
admin_gdf = gpd.read_file(admin_shp_path)
# projected_crs = 'EPSG:4326' #Untuk sistem referensi koordinat WGS 84 (geografi):
# projected_crs = 'EPSG:32648' #Untuk Universal Transverse Mercator (UTM) Zone 48S (tergantung pada zona UTM yang diinginkan):
# projected_crs = 'EPSG:3857' #Untuk Web Mercator (digunakan oleh OpenStreetMap, Google Maps, dan lainnya):
# admin_gdf = admin_gdf.to_crs(projected_crs)
# Hitung titik tengah dan konversi ke lon-lat
admin_gdf['centroid_latitude'] = admin_gdf['geometry'].centroid.y
admin_gdf['centroid_longitude'] = admin_gdf['geometry'].centroid.x
admin_gdf_per_distrik = admin_gdf.dissolve(by='Distrik').reset_index()

/var/folders/b9/v73gyxys1djbgjy3c8c3cfwm0000gn/T/ipykernel_55075/1426966402.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  admin_gdf['centroid_latitude'] = admin_gdf['geometry'].centroid.y
/var/folders/b9/v73gyxys1djbgjy3c8c3cfwm0000gn/T/ipykernel_55075/1426966402.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  admin_gdf['centroid_longitude'] = admin_gdf['geometry'].centroid.x


In [20]:
# sagu
sagu_gdf = gpd.read_file('../data/SHP/Daerah_Sagu.shp')
# projected_crs = 'EPSG:3857' #Untuk Web Mercator (digunakan oleh OpenStreetMap, Google Maps, dan lainnya):
# sagu_gdf = sagu_gdf.to_crs(projected_crs)
sagu_gdf['centroid_latitude'] = sagu_gdf['geometry'].centroid.y
sagu_gdf['centroid_longitude'] = sagu_gdf['geometry'].centroid.x
# sagu_gdf = sagu_gdf.dissolve(by='Penggunaan').reset_index()

# Plotly

In [21]:
# # hitung titik tengah peta
# titik_titik=admin_gdf['geometry'].centroid.to_list()
# multi_point = MultiPoint(titik_titik)
# titik_tengah = multi_point.centroid
# # tampilkan hasil
# fig = px.choropleth_mapbox(
#     admin_gdf,
#     locations=admin_gdf.index,
#     geojson=admin_gdf.geometry,
#     color="Distrik",
#     hover_name="Kampung",
#     hover_data=['Provinsi', 'Kabupaten', 'Distrik', 'Kampung', 'Sumber'],
#     mapbox_style="open-street-map",
#     center={"lat": titik_tengah.y, "lon": titik_tengah.x},
#     zoom=10.8,
#     opacity=0.2,
# )

# # Adjust layout
# fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
# fig.write_html('../map-html/map_plotly.html')

# Folium

In [22]:
# Ambil titik-titik dari kolom 'geometry'
titik_titik = admin_gdf['geometry'].centroid.to_list()
multi_point = MultiPoint(titik_titik)
titik_tengah = multi_point.centroid

# Function to generate a random color
def random_color():
    return '#{:06x}'.format(random.randint(0, 0xFFFFFF))

# Create a dictionary mapping each KECEMATAN to a random color
unique_distrik = admin_gdf['Distrik'].unique()
color_dict = {distrik: random_color() for distrik in unique_distrik}

# Buat peta folium
m = folium.Map(location=[titik_tengah.y, titik_tengah.x], zoom_start=9, tiles='openstreetmap')

# Tambahkan GeoJSON ke peta folium
folium.GeoJson(
    admin_gdf_per_distrik,
    name='Batas Administrasi Distrik',
    style_function = lambda feature: {
        'fillColor': color_dict[feature['properties']['Distrik']],
        'color': 'black',
        'weight': 0.3,
        'lineJoin': 'round',
    },
    # highlight_function=lambda feature: {
    #     "color": "#0c7e54",
    #     'weight': 3,
    # },
    smooth_factor=2,
    ).add_to(m)

# Tambahkan GeoJSON ke peta folium
folium.GeoJson(
    admin_gdf,
    name='Batas Administrasi Kampung',
    style_function = lambda feature: {
        'fillColor': color_dict[feature['properties']['Distrik']],
        'color': 'black',
        'weight': 0.5,
        'lineJoin': 'round',
        'dashArray': '5, 5',
    },
    highlight_function=lambda feature: {
        "color": "#0c7e54",
        'weight': 3,
        'dashArray': None,  # Set 'dashArray' menjadi None untuk garis lurus saat di-highlight
    },
    # tooltip=folium.GeoJsonTooltip(
    #     fields=['Kampung', 'Distrik', 'Kabupaten', 'Provinsi', 'Sumber'],
    #     sticky=False,
    # ),
    popup=folium.GeoJsonPopup(
        fields=['Kampung', 'Distrik', 'Kabupaten', 'Provinsi', 'Sumber'],
        aliases=['Kampung', 'Distrik', 'Kabupaten', 'Provinsi', 'Sumber'],
        localize=True,
        labels=True,
    ),
    smooth_factor=2,
    ).add_to(m)


titik_group = folium.FeatureGroup("Titik Kampung", show=True).add_to(m)
# Tambahkan label kampung pada setiap shape
for idx, row in admin_gdf.iterrows():
    folium.Marker(
        location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
        # tooltip=folium.Tooltip(  # Change Popup to Tooltip
        #     f"""
        #         <b>Kampung:</b> {row['Kampung']}<br>
        #         <b>Distrik:</b> {row['Distrik']}<br>
        #         <b>Kabupaten:</b> {row['Kabupaten']}<br>
        #         <b>Provinsi:</b> {row['Provinsi']}<br>
        #         <b>Sumber:</b> {row['Sumber']}<br>
        #     """,
        #     sticky=False
        # ),
        # icon=folium.Icon(color='blue', icon='info-sign')
        icon=folium.DivIcon(html=f"""<div style="font-size: 10pt; color: black">{row['Kampung']}</div>"""),
    ).add_to(titik_group)

titik_distrik_group = folium.FeatureGroup("Titik Distrik", show=True).add_to(m)
# Tambahkan label kampung pada setiap shape
for idx, row in admin_gdf_per_distrik.iterrows():
    folium.Marker(
        location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
        # tooltip=folium.Tooltip(  # Change Popup to Tooltip
        #     f"""
        #         <b>Distrik:</b> {row['Distrik']}<br>
        #         <b>Kabupaten:</b> {row['Kabupaten']}<br>
        #         <b>Provinsi:</b> {row['Provinsi']}<br>
        #         <b>Sumber:</b> {row['Sumber']}<br>
        #         <b>Color:</b> {color_dict[row['Distrik']]}<br>
        #     """,
        #     sticky=False
        # ),
        icon=folium.DivIcon(html=f"""<div style="font-size: 14pt; font-weight:bold; text-shadow: 1px 1px 1px #ffffff; color: {color_dict[row['Distrik']]}">{row['Distrik']}</div>"""),
    ).add_to(titik_distrik_group)

stripes = plugins.pattern.StripePattern(angle=-45).add_to(m)
folium.GeoJson(
    sagu_gdf,
    name='Area Hutan Sagu',
    style_function=lambda feature: {
        "fillColor": "#009b00",
        "color": "#008200",
        "fillPattern": stripes,
        "weight": 0.5,
        "fillOpacity": 0.7,
        'lineJoin': 'round',
    },
    highlight_function=lambda feature: {
        "fillColor": "#0c7e54",
        "fillPattern": None,
        # 'weight': 3,
    },
    tooltip=folium.Tooltip(  # Change Popup to Tooltip
        f"""
            Area hutan sagu
        """,
        sticky=True
    ),
    # popup=folium.GeoJsonPopup(
    #     fields=['Penggunaan'],
    #     aliases=['Penggunaan'],
    #     localize=True,
    #     labels=True,
    # ),
    smooth_factor=2,
    ).add_to(m)

# legend_group = folium.FeatureGroup("Legend").add_to(m)
# Buat HTML custom untuk legenda berdasarkan color_dict
# legend_html = """
# <div style="position: fixed; 
#              bottom: 30px; right: 10px; width: 200px; height: auto; 
#              border:2px solid grey; z-index:9999; font-size:14px;
#              background-color:white;
#              ">&nbsp; Distrik <br>
# """
# for key, value in color_dict.items():
#     legend_html += f'&nbsp; {key} &nbsp; <i class="fa fa-square" style="color:{value}"></i><br>'
# legend_html += "</div>"
# # Tambahkan HTML custom sebagai legenda ke peta
# m.get_root().html.add_child(folium.Element(legend_html))

plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)
folium.TileLayer("cartodbpositron").add_to(m)
folium.TileLayer("esri-worldimagery").add_to(m)
folium.LayerControl().add_to(m)

/var/folders/b9/v73gyxys1djbgjy3c8c3cfwm0000gn/T/ipykernel_55075/2841927141.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  titik_titik = admin_gdf['geometry'].centroid.to_list()


In [23]:
# Tampilkan peta
m.save('../map-html/map_folium.html')
# m

In [24]:
color_dict

{'Aranday': '#9c6c0c',
 'Aroba': '#e15533',
 'Babo': '#9c558c',
 'Bintuni': '#752517',
 'Biscoop': '#4208e7',
 'Dataran Beimes': '#5fe70c',
 'Fafurwar': '#c88e51',
 'Kaitaro': '#87b9cf',
 'Kamundan': '#8f3147',
 'Kuri': '#1ccbbe',
 'Manimeri': '#294e81',
 'Masyeta': '#a393b1',
 'Merdey': '#26994d',
 'Meyado': '#436039',
 'Moskona Barat': '#46b2d5',
 'Moskona Selatan': '#d7c563',
 'Moskona Timur': '#25a1e2',
 'Moskona Utara': '#a66457',
 'Sumuri': '#7d6873',
 'Tembuni': '#0df264',
 'Tomu': '#4f4dc5',
 'Tuhiba': '#3cbaff',
 'Wamesa': '#9bff56',
 'Weriagar': '#51b02a'}